# Enhancing Medical Report Findings with Retrieval-Augmented Generation (RAG): Integrating LLM Models and Chroma DB using the LangChain framework for searchable data.

### Installing the required libraries

In [1]:
!pip install "PyPDF2" "chromadb==0.4.0" "langchain==0.2.0" "langchain-community" "sentence-transformers" 

  Using cached langchain-0.2.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached pydantic-1.10.19-cp312-cp312-win_amd64.whl.metadata (153 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached SQLAlchemy-2.0.36-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.11.11-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)


### Reading PDF reports and remove sensitive data

In [1]:
from PyPDF2 import PdfReader
def read_pdf(file_path):
    # Initialize the reader for the PDF file
    
    reader = PdfReader(file_path)
    
    # Extract text from each page and store it in a variable
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text() + "\n"  # Adding a newline for separation between pages
  
    text = pdf_text.replace("\n", " ")
    
    return text
    #return page

In [2]:
import re
def remove_sensitive(text):
    text = re.sub(r"Patient\s+Name\s+:\s+.*?\s+(?=Patient\s+ID)", "", text)#removing patient name
    text = re.sub(r"Referring\s+Physician\s+.*?\s+Report", "Report", text)
    text = re.sub(r"Dr [A-Za-z\s]+MD\s*|Reg No:\s*\d+", "", text).strip()
    return text


### Creating field for json and removing other sensitive data

In [3]:
import re
import json
def text_json(extracted_data):
# Regex patterns to extract fields
    patterns = {
        "Patient ID": r"Patient\s+ID\s*:\s*(\S+)",
        "Date": r"Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})",
        "Time": r"(\d{2}:\d{2}:\d{2})",
        "Patient age": r"Patient\s+age\s+/\s+Sex\s+:\s+(\d{3}Y)",
        "Sex": r"Sex\s*:\s*\d{3}[A-Za-z]*\s*/\s*(\w)",
        "TECHNIQUE": r"TECHNIQUE\s*:\s*(.*?)\s*FINDINGS",
        "FINDINGS": r"FINDINGS\s*:\s*(.*?)\s*IMPRESSION",
        "IMPRESSION": r"IMPRESSION\s*:\s*(.*)"
    }
    
    # Extract fields using regex
    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, extracted_data, re.DOTALL)
        if match:
            data[key] = match.group(1).strip()
    
    # removes the unwanted portion from the "FINDINGS" and "IMPRESSION" field while keeping the rest of the content intact.
    try:
        findings = data["FINDINGS"]
        impression = data["IMPRESSION"]
        updated_findings = re.sub(r"Patient\s+ID\s*:\s*(\S+).*Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})\s*(\S+)", "", findings).strip()
        updated_impression = re.sub(r"Patient\s+ID\s*:\s*(\S+).*Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})\s*(\S+)", "", impression).strip()

        # Update the Data
        data["FINDINGS"] = updated_findings
        data["IMPRESSION"] = updated_impression
    except:
        print("no unwanted found")
    # Update the JSON
    json_data = json.dumps(data, indent=4)
    
    return json_data
    

In [4]:
# Define the file path to the PDF file named "MO.pdf" in the current working directory
file_path = r"Data/MO.pdf"  # The 'r' prefix indicates a raw string to handle any special characters in the path

# Read the PDF file and extract its content as text
print(
    # Remove sensitive information like patient name, physician name, etc.
    text_json(
        remove_sensitive(
            read_pdf(file_path)  # Read PDF and extract text content
        )
    )
)
# Convert the sanitized text to a structured JSON format and print it

{
    "Patient ID": "MR0000444444",
    "Date": "20 April  2020",
    "Time": "10:16:04",
    "Patient age": "075Y",
    "Sex": "M",
    "TECHNIQUE": "T2 FSE Axials  / Sagittals  & Coronals.   T1 &T2  FLAIR  Axials.  DWI,  GRE  Axials.    3D TOF  MR Angiography  of Intracranial  Arteries.",
    "FINDINGS": "Large wedge shaped lesion with restricted diffusion low ADC values and hyperintensities  in T2 & FLAIR images noted involving capsuloganglionic region, parietal lobe, adjacent  frontal insular cortex, temporal lobe in right side.   Focal FLAIR hyperintensities without restricted diffusion in bilateral posterior  periventricular wh ite matter, left corona radiata and centrum semiovale.   Curvilinear  blooming  SWI hypointensities  in distal  right  MCA.   Cerebellum, 4th ventricle, brain stem & CP angle regions are within normal limits.  Sella, suprasellar & parasellar areas are normal.   The ex tracerebral spaces and supratentorial ventricular system are normal.  Rest of the cerebra

### Processing multiple pdf file for chromadb format

In [5]:
# Process all PDF files in the folder
import os
folder_path = r"Data/"
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        print(pdf_path)
        extracted_data = read_pdf(pdf_path)
        remove_sensitive_data = remove_sensitive(extracted_data)
        extracted_json = text_json(remove_sensitive_data)
        ## Parse the JSON strings into dictionaries and append to the list
        all_data.append(json.loads(extracted_json))
#print(all_data)

Data/MO.pdf
Data/RA.pdf
Data/VA.pdf


### Converting JSON to ChromaDB format

In [6]:
from langchain.schema import Document

documents = [
    Document(page_content=f"{item['Patient ID']} {item['Patient age']} {item['Sex']} {item['TECHNIQUE']} {item['FINDINGS']} {item['IMPRESSION']}", metadata={})
    for item in all_data
]


In [7]:
# Verify the result
for doc in documents:
    print(f"Page Content: {doc.page_content}, Metadata: {doc.metadata}")

Page Content: MR0000444444 075Y M T2 FSE Axials  / Sagittals  & Coronals.   T1 &T2  FLAIR  Axials.  DWI,  GRE  Axials.    3D TOF  MR Angiography  of Intracranial  Arteries. Large wedge shaped lesion with restricted diffusion low ADC values and hyperintensities  in T2 & FLAIR images noted involving capsuloganglionic region, parietal lobe, adjacent  frontal insular cortex, temporal lobe in right side.   Focal FLAIR hyperintensities without restricted diffusion in bilateral posterior  periventricular wh ite matter, left corona radiata and centrum semiovale.   Curvilinear  blooming  SWI hypointensities  in distal  right  MCA.   Cerebellum, 4th ventricle, brain stem & CP angle regions are within normal limits.  Sella, suprasellar & parasellar areas are normal.   The ex tracerebral spaces and supratentorial ventricular system are normal.  Rest of the cerebral parenchyma is normal is normal.   Midline structures and corpus callosum are normal.  No haemorrhagic pathology.   No extraaxial  coll

### Creating chunks from document from chromaDB (vector database)
### Using all-MiniLM-L6-v2 embedings
### Saving the data into chromaDB

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# -------------------------------
# Split Data into Chunks (Optional for Large Content)
# -------------------------------
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

# -------------------------------
# Use Hugging Face Embeddings (all-MiniLM-L6-v2)
# -------------------------------
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# -------------------------------
# Store Embeddings in ChromaDB
# -------------------------------
chroma_db4 = Chroma.from_documents(split_documents, embeddings, persist_directory="./chroma_sample_db")

# Save embeddings to the ChromaDB directory
chroma_db4.persist()
print("Data has been successfully inserted into ChromaDB!")

Data has been successfully inserted into ChromaDB!


In [9]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


### Test Queries

In [10]:
def retrieve_queries(queries, k):
    # Create a retriever instance from the Chroma database to retrieve relevant documents
    retriever = chroma_db4.as_retriever()
    # Loop through each query in the test_queries list
    for query in queries:
        # Retrieve the top k most relevant documents for the current query
        results = retriever.get_relevant_documents(query, k=k)
        
        # Print the current query being tested
        print(f"🔍 Query: {query}")
        
        # Loop through the top k results and display their content
        for i, doc in enumerate(results[:k]):  # Limit to top k results
            print(f"💡 Result {i+1}: {doc.page_content}")
        
        # Print a separator line for better readability between results
        print("-" * 50)

In [11]:
# Define a list of test queries to be processed
queries = ["Patient is not cooperative"]
k = 1
retrieve_queries(queries, k)

C:\Users\bhowm\.conda\envs\new1\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


🔍 Query: Patient is not cooperative
💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
--------------------------------------------------


In [15]:
# Define a list of test queries to be processed
queries = ["including right side of splenium of corpus callosum"]
k = 2
retrieve_queries(queries, k)

🔍 Query: including right side of splenium of corpus callosum
💡 Result 1: IP0000111111 067Y M T1, T2 & FLAIR  Axials,T2  Saggitals  / Coronals.  DWI  Axials.   Clinical  details  : S/p CABG  for CAD. Evidence of multiple lacunar &  tiny sized lesions with restricted diffusion and low ADC  values are hyperintense on FLAIR images noted in both cerebellar hemispheres, occipital  lobes and parietooccipital regions (including right side of splenium of corpus callosum),  left posterior periv entricular white matter, left posterior temporal lobe, left frontal corona  radiata
💡 Result 2: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         F

# bleu_score

BLEU (Bilingual Evaluation Understudy) is a metric for evaluating the quality of machine-translated text by comparing it to a reference text. It measures n-gram overlap (word sequences) between the generated and reference texts.


BLEU Score	Interpretation
0.9 - 1.0	Perfect translation
0.7 - 0.9	High-quality translation
0.4 - 0.7	Moderate quality
0.1 - 0.4	Weak translation
0.0 - 0.1	Poor match


In [13]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure NLTK components are available
nltk.download('punkt')

# Define a reference document (ground truth)
reference_text = "Patient is not cooperative"
reference_tokens = [nltk.word_tokenize(reference_text.lower())]  # Tokenize & lowercase

# Test Query
test_query = "Patient is non-cooperative"

# Retrieve relevant documents
retriever = chroma_db4.as_retriever()
results = retriever.get_relevant_documents(test_query, k=2)

# Compute BLEU Score for each retrieved document
for i, doc in enumerate(results[:2]):  # Top 2 results
    candidate_tokens = nltk.word_tokenize(doc.page_content.lower())  # Tokenize candidate
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, 
                               smoothing_function=SmoothingFunction().method1)

    print(f"💡 Result {i+1}: {doc.page_content}")
    print(f"🔹 BLEU Score: {bleu_score:.4f}")  # Print BLEU score
    print("-" * 50)

💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
🔹 BLEU Score: 0.0278
--------------------------------------------------
💡 Result 2: MCA branches are not well visualized, possible due to slow flow  status / occlusion.   * Small  stenotic  segment  in distal  left MCA.   * Mild  stenosis  in cavernous  and infrapetrous  parts  of right  ICA.   * Hypoplastic  right  vertebral  artery.       Consultant Radiologist
🔹 BLEU Score: 0.0041
--------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhowm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# rouge_scorer

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate text summarization and machine-generated content by comparing it to a reference text. Unlike BLEU (which focuses on precision), ROUGE prioritizes recall, measuring how much of the reference text appears in the generated text.


ROUGE Score Interpretation
ROUGE Score	Interpretation
0.8 - 1.0	Excellent summary
0.6 - 0.8	Good summary
0.4 - 0.6	Moderate similarity
0.2 - 0.4	Weak match
0.0 - 0.2	Poor summary

In [26]:
pip install rouge-score nltk


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=f7398977d5c58eae4abc4d44709d2ac91c5f348596530b43aeb9ea989707d827
  Stored in directory: c:\users\bhowm\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [27]:
import nltk
from rouge_score import rouge_scorer

# Define a reference document (ground truth)
reference_text = "Patient is not cooperative"

# Test Query
test_query = "Patient is non-cooperative"

# Retrieve relevant documents
retriever = chroma_db4.as_retriever()
results = retriever.get_relevant_documents(test_query, k=2)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE Score for each retrieved document
for i, doc in enumerate(results[:2]):  # Top 2 results
    scores = scorer.score(reference_text, doc.page_content)

    print(f"💡 Result {i+1}: {doc.page_content}")
    print(f"🔹 ROUGE-1 (Unigrams) Recall: {scores['rouge1'].recall:.4f}, Precision: {scores['rouge1'].precision:.4f}, F1: {scores['rouge1'].fmeasure:.4f}")
    print(f"🔹 ROUGE-2 (Bigrams) Recall: {scores['rouge2'].recall:.4f}, Precision: {scores['rouge2'].precision:.4f}, F1: {scores['rouge2'].fmeasure:.4f}")
    print(f"🔹 ROUGE-L (Longest Common Subsequence) Recall: {scores['rougeL'].recall:.4f}, Precision: {scores['rougeL'].precision:.4f}, F1: {scores['rougeL'].fmeasure:.4f}")
    print("-" * 50)


💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
🔹 ROUGE-1 (Unigrams) Recall: 1.0000, Precision: 0.0588, F1: 0.1111
🔹 ROUGE-2 (Bigrams) Recall: 1.0000, Precision: 0.0448, F1: 0.0857
🔹 ROUGE-L (Longest Common Subsequence) Recall: 1.0000, Precision: 0.0588, F1: 0.1111
--------------------------------------------------
💡 Result 2: MCA branches are not well visualized, possible due to slow flow  status / occlusion.   * Small  stenotic  segment  in distal  left MCA.   * Mild  stenosis  in cavernous  and infrapetrous  parts  of right  ICA.  

# METEOR Score

METEOR (Metric for Evaluation of Translation with Explicit ORdering) is a text evaluation metric designed to improve upon BLEU and ROUGE by considering:
Synonyms & Stemming (word variations count as matches)
Word Order (penalizes incorrect word sequence)
Recall & Precision Balance (avoids BLEU’s bias toward precision)

METEOR Score	Interpretation
0.9 - 1.0	Perfect match
0.7 - 0.9	High similarity
0.4 - 0.7	Moderate match
0.2 - 0.4	Weak similarity
0.0 - 0.2	Poor match


METEOR vs BLEU vs ROUGE
Metric	Strengths	Weaknesses
BLEU	Good for machine translation	Ignores synonyms, sensitive to word order
ROUGE	Good for summarization	Doesn't account for meaning
METEOR	Best for paraphrased sentences	Slightly slower computation

In [28]:
import nltk
from nltk.translate.meteor_score import meteor_score

# Ensure NLTK components are available
nltk.download('wordnet')
nltk.download('punkt')

# Define a reference document (ground truth)
reference_text = "Patient is not cooperative"
reference_tokens = nltk.word_tokenize(reference_text.lower())  # Tokenize & lowercase

# Test Query
test_query = "Patient is non-cooperative"

# Retrieve relevant documents
retriever = chroma_db4.as_retriever()
results = retriever.get_relevant_documents(test_query, k=2)

# Compute METEOR Score for each retrieved document
for i, doc in enumerate(results[:2]):  # Top 2 results
    candidate_tokens = nltk.word_tokenize(doc.page_content.lower())  # Tokenize candidate
    meteor = meteor_score([reference_tokens], candidate_tokens)  # Compute METEOR

    print(f"💡 Result {i+1}: {doc.page_content}")
    print(f"🔹 METEOR Score: {meteor:.4f}")  # Print METEOR score
    print("-" * 50)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhowm\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhowm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
🔹 METEOR Score: 0.2698
--------------------------------------------------
💡 Result 2: MCA branches are not well visualized, possible due to slow flow  status / occlusion.   * Small  stenotic  segment  in distal  left MCA.   * Mild  stenosis  in cavernous  and infrapetrous  parts  of right  ICA.   * Hypoplastic  right  vertebral  artery.       Consultant Radiologist
🔹 METEOR Score: 0.0617
--------------------------------------------------


# bert-score

BERTScore is a modern text evaluation metric that compares deep semantic meaning rather than just word overlap. It uses BERT (or similar Transformer models) to generate word embeddings and calculates similarity between the generated and reference texts.

BERTScore Interpretation
BERTScore	Interpretation
0.9 - 1.0	Almost identical
0.7 - 0.9	Strong similarity
0.4 - 0.7	Moderate match
0.2 - 0.4	Weak similarity
0.0 - 0.2	Poor match


BERTScore vs BLEU vs ROUGE
Metric	Strengths	Weaknesses
BLEU	Good for exact word overlap	Fails on paraphrased text
ROUGE	Good for summarization	Ignores meaning differences
BERTScore	Captures semantic meaning & synonyms	Requires deep learning models (slower)


In [30]:
pip install bert-score

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.0 MB 21.8 MB/s eta 0:00:01
   ------------------ --------------------- 3.7/8.0 MB 21.8 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.0 MB 9.5 MB/s eta 0:00:01
   --------------------------------- ------ 6.8/8.0 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------------------- ---------------- 1.3/2.2 MB 8.4 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:
import torch
from bert_score import score

# Define reference document (ground truth)
reference_text = "Patient is not cooperative"

# Test Query
test_query = "Patient is non-cooperative"

# Retrieve relevant documents
retriever = chroma_db4.as_retriever()
results = retriever.get_relevant_documents(test_query, k=2)

# Compute BERTScore for each retrieved document
for i, doc in enumerate(results[:2]):  # Top 2 results
    candidate_text = doc.page_content  # Retrieved text

    # Compute BERTScore (Precision, Recall, F1)
    P, R, F1 = score([candidate_text], [reference_text], lang="en", model_type="microsoft/deberta-xlarge-mnli")

    print(f"💡 Result {i+1}: {doc.page_content}")
    print(f"🔹 BERTScore Precision: {P.item():.4f}, Recall: {R.item():.4f}, F1 Score: {F1.item():.4f}")
    print("-" * 50)


C:\Users\bhowm\.conda\envs\new1\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhowm\.cache\huggingface\hub\models--microsoft--deberta-xlarge-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
🔹 BERTScore Precision: 0.3465, Recall: 0.7349, F1 Score: 0.4710
--------------------------------------------------
💡 Result 2: MCA branches are not well visualized, possible due to slow flow  status / occlusion.   * Small  stenotic  segment  in distal  left MCA.   * Mild  stenosis  in cavernous  and infrapetrous  parts  of right  ICA.   * Hypoplastic  right  vertebral  artery.       Consultant Radiologist
🔹 BERTScore Precision: 0.3286, Recall: 0.5799, F1 Score: 0.4195
-------------------

## Summerization

In [13]:
pip install ollama

  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl (2.0 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.0 requires pydantic<2.0,>=1.9, but you have pydantic 2.10.6 which is incompatible.
fastapi 0.99.1 requires pydantic!=1.8,!=1.8.1,<2.0.0,>=1.7.4, but you have pydantic 2.10.6 which is incompatible.


In [15]:
pip install --upgrade pydantic

In [2]:
pip uninstall pydantic -y

Found existing installation: pydantic 1.10.13
Uninstalling pydantic-1.10.13:
  Successfully uninstalled pydantic-1.10.13
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pydantic==1.10.13

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama 0.4.7 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 1.10.13 which is incompatible.



  Using cached pydantic-1.10.13-py3-none-any.whl.metadata (149 kB)
Using cached pydantic-1.10.13-py3-none-any.whl (158 kB)


In [2]:
import ollama

def summarize_medical_report(question, answer):
    prompt = f"""
    Given the medical report and patient condition:

    Question: "{question}"
    Answer: "{answer}"

    **Task:** Summarize the key medical findings, focusing on abnormalities. If the patient was non-cooperative, mention any limitations in the findings.

    **Output Format:** 
    - Key abnormalities only
    - Impact of patient non-cooperation
    - Concise and medically relevant summary

    **Example Output:**
    - Due to patient non-cooperation, distal right MCA branches were not well visualized.
    - Small stenotic segment in distal left MCA.
    - Mild stenosis in cavernous and infrapetrous parts of right ICA.
    - Fetal-type A1 segment of right ACA noted as a normal variant.
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]




In [3]:
# Example Usage
question = "Patient is non-cooperative"
answer = """are normal.  Rest of the cerebral parenchyma is normal.   
Midline  structures  and corpus  callosum  are normal.   
No haemorrhagic pathology. No extraaxial collection seen.  
MR Angiography shows : (Patient is not cooperative)   
Distal right MCA branches were not well visualized.   
Small  stenotic  segment  in distal  left MCA.   
Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         
Fetal  type of A1 segment of right ACA  -- Normal  varient.   
Hypoplastic right"""

summary = summarize_medical_report(question, answer)
print(summary)

 - Due to patient non-cooperation, visualization of the distal right MCA branches is limited. However, a small stenotic segment in the distal left MCA and mild stenosis in both cavernous and infrapetrous parts of the right ICA are noted. The A1 segment of the right ACA was found to be of fetal type, which is considered a normal variant.
